# 請求書自動作成

## 1. ライブラリ・データ読み込み

In [1]:
import pandas as pd
import openpyxl, pprint, os, sys
from datetime import datetime, timedelta
from openpyxl.styles import Alignment
from dateutil.relativedelta import relativedelta

In [2]:
# csvファイルを読み込む
products = pd.read_csv("../input/products.csv", encoding="shift_jis")
orders = pd.read_csv("../input/orders.csv", encoding="shift_jis")

In [3]:
products.head()

,product_id,product_name,unit_price,unit,note,tax_rate,category,supplier
0,P001,ノートパソコン,100000,台,15.6インチ SSD 512GB,10,PC本体,TechSupplier A
1,P002,レーザープリンタ,40000,台,両面印刷対応 A4,10,周辺機器,OfficeGear Inc.
2,P003,インクカートリッジ,3000,個,ブラック・カラー対応,10,消耗品,InkWorks
3,P004,HDMIケーブル,1200,本,1.5m 金メッキ仕様,10,周辺機器,CableMart
4,P005,USBメモリ 64GB,2000,本,USB 3.0対応,10,周辺機器,MemoryWorld


In [4]:
orders.head()

,order_id,customer_name,order_date,product_id,quantity,invoice_registration_number
0,O0001,高橋真由美,2025/5/10,P007,5,T1234567890001
1,O0001,高橋真由美,2025/5/10,P008,5,T1234567890001
2,O0001,高橋真由美,2025/5/10,P004,5,T1234567890001
3,O0002,鈴木健,2025/5/11,P010,1,T1234567890002
4,O0002,鈴木健,2025/5/11,P008,5,T1234567890002


In [5]:
# テンプレートを読み込む
invoice_wb = openpyxl.load_workbook("../template/invoice_template.xlsx")
# 対象のワークシートを読み込む
invoice_ws = invoice_wb["Invoice"]

# 2. 注文IDを入力し、対象のデータを取り出す

In [6]:
# 対象の注文IDを対話形式で入力
target_id = input("注文IDを入力して下さい")

注文IDを入力して下さいO0001


In [7]:
# 情報を元に見積書を作成
# 対象の取引データを抽出
target_orders = orders[orders["order_id"] == target_id]
target_orders

,order_id,customer_name,order_date,product_id,quantity,invoice_registration_number
0,O0001,高橋真由美,2025/5/10,P007,5,T1234567890001
1,O0001,高橋真由美,2025/5/10,P008,5,T1234567890001
2,O0001,高橋真由美,2025/5/10,P004,5,T1234567890001


In [8]:
# データを結合
merged = target_orders.merge(products, on="product_id", how="left")
merged

,order_id,customer_name,order_date,product_id,quantity,invoice_registration_number,product_name,unit_price,unit,note,tax_rate,category,supplier
0,O0001,高橋真由美,2025/5/10,P007,5,T1234567890001,液晶モニター 24インチ,18000,台,IPS 非光沢 HDMI対応,10,周辺機器,Display Tech
1,O0001,高橋真由美,2025/5/10,P008,5,T1234567890001,キーボード（日本語配列）,3500,個,USB接続タイプ,10,周辺機器,Keyboards Ltd.
2,O0001,高橋真由美,2025/5/10,P004,5,T1234567890001,HDMIケーブル,1200,本,1.5m 金メッキ仕様,10,周辺機器,CableMart


In [9]:
# 金額列の小計
merged["subtotal"]  = merged["unit_price"] * merged["quantity"]

In [10]:
# 消費税
merged["VAT"] = merged["subtotal"] * merged["tax_rate"] / 100
merged["VAT"] = merged["VAT"].round(0).astype(int)

In [11]:
# 合計
merged["total_amount"] = merged["subtotal"] + merged["VAT"]
merged["total_amount"] = merged["total_amount"].astype(int)

In [12]:
merged

,order_id,customer_name,order_date,product_id,quantity,invoice_registration_number,product_name,unit_price,unit,note,tax_rate,category,supplier,subtotal,VAT,total_amount
0,O0001,高橋真由美,2025/5/10,P007,5,T1234567890001,液晶モニター 24インチ,18000,台,IPS 非光沢 HDMI対応,10,周辺機器,Display Tech,90000,9000,99000
1,O0001,高橋真由美,2025/5/10,P008,5,T1234567890001,キーボード（日本語配列）,3500,個,USB接続タイプ,10,周辺機器,Keyboards Ltd.,17500,1750,19250
2,O0001,高橋真由美,2025/5/10,P004,5,T1234567890001,HDMIケーブル,1200,本,1.5m 金メッキ仕様,10,周辺機器,CableMart,6000,600,6600


## 3. 請求書作成、ファイルを出力

In [13]:
# 請求書への入力作業
# 明細の開始行
start_row = 15

# 右詰めスタイルを作成
right_align = Alignment(horizontal="right")

# 日付設定
today = datetime.today()
next_month_same_day = today + relativedelta(months=1)

# 明細行描写
for i, (_, row) in enumerate(merged.iterrows()):
    # No(連番)
    invoice_ws.cell(row=start_row + i, column=1, value=i + 1) 
    # 商品名
    invoice_ws.cell(row=start_row + i, column=2, value=row["product_name"]) 
     # 数量
    invoice_ws.cell(row=start_row + i, column=3, value=row["quantity"])
    # 単価(円)
    cell = invoice_ws.cell(row=start_row + i, column=4, value=row["unit_price"]) 
    cell.number_format = '"¥"#,##0'
    # 税率
    invoice_ws.cell(row=start_row + i, column =5, value=str(row["tax_rate"])+"%")
    # 金額(円)
    cell = invoice_ws.cell(row=start_row + i, column=6, value=row["subtotal"]) 
    cell.number_format = '"¥"#,##0'
    
# 小計
cell = invoice_ws.cell(row=31, column=6, value=merged["subtotal"].sum()) 
cell.number_format = '"¥"#,##0'

# 消費税
cell = invoice_ws.cell(row=32, column=6, value=merged["VAT"].sum()) 
cell.number_format = '"¥"#,##0'

# 合計金額
cell = invoice_ws.cell(row=33, column=6, value=merged["total_amount"].sum()) 
cell.number_format = '"¥"#,##0'

# 請求金額
amount = merged["total_amount"].sum()
formatted = f"¥{amount:,.0f}(税込)"

cell = invoice_ws.cell(row=12, column=3, value=formatted)
cell.alignment = right_align

# 税率区分
# 10%
cell = invoice_ws.cell(row=36, column=4, 
                         value=merged[merged["tax_rate"] == 10]["VAT"].sum())
cell.number_format = '"¥"#,##0'

cell = invoice_ws.cell(row=36, column=5, 
                         value=merged[merged["tax_rate"] == 10]["subtotal"].sum())
cell.number_format = '"¥"#,##0'
cell.alignment = right_align

# 8%
cell = invoice_ws.cell(row=37, column=4, 
                         value=merged[merged["tax_rate"] == 8]["VAT"].sum())
cell.number_format = '"¥"#,##0'

cell = invoice_ws.cell(row=37, column=5, 
                         value=merged[merged["tax_rate"] == 8]["subtotal"].sum())
cell.number_format = '"¥"#,##0'
cell.alignment = right_align

# 宛名
invoice_ws["A4"] = merged.iloc[0]["customer_name"] + "  様"

# 発行日
cell = invoice_ws.cell(row=5, column=6, value=today.strftime("%Y年%m月%d日")) 
cell.alignment = right_align

# 登録番号
cell = invoice_ws.cell(row=6, column=6, 
                         value=merged["invoice_registration_number"].iloc[0]) 
cell.alignment = right_align

# 請求番号
cell = invoice_ws.cell(row=7, column=6, 
                         value=merged["order_id"].iloc[0]) 
cell.alignment = right_align

# お支払期日
cell = invoice_ws.cell(row=42, column=3, 
                         value=next_month_same_day.strftime("%Y年%m月末"))
cell.alignment = right_align
    
invoice_wb.save(f"../output/{target_id}_invoice.xlsx")